# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.madras_laftr.models import *
from models.madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 100
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
cv_seeds = [13, 29, 42, 55, 73]

## Load data

In [5]:
data_name = 'adult-race'

In [6]:
x, y, a = load_data(data_name)
raw_data = (x, y, a)

In [7]:
xdim = x.shape[1]
ydim = y.shape[1]
adim = a.shape[1]
zdim = 8

## Result file

In [8]:
header = "model_name", "cv_seed", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [9]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)    
    
    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        
        clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0725230872631073 | 0.3525453805923462 | 0.6748806238174438 | 0.9193267822265625 | 0.7255187246963563 | 0.791023532388664 | 0.02634741902834008
> 2 | 0.06398490071296692 | 0.32769912481307983 | 0.6787222623825073 | 0.9193267822265625 | 0.8338815789473685 | 0.8610513663967612 | 0.02634741902834008
> 3 | 0.06155753508210182 | 0.3191608786582947 | 0.6724411845207214 | 0.9193267822265625 | 0.837171052631579 | 0.8610513663967612 | 0.02634741902834008
> 4 | 0.0607931911945343 | 0.3163074553012848 | 0.6696392297744751 | 0.9193267822265625 | 0.8396381578947368 | 0.8610513663967612 | 0.02634741902834008
> 5 | 0.0607309527695179 | 0.3151129186153412 | 0.6646001935005188 | 0.9193267822265625 | 0.8404605263157895 | 0.8610513663967612 | 0.02634741902834008
> 6 | 0.060735542327165604 | 0.314389705657959 | 0.6609154343605042 | 0.9193267822265625 | 0.8421685222672065 | 0.8610513663967612 | 0.0263474190283400

### LAFTR for Eq Odds

In [10]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)    
    
    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        
        clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.11155003309249878 | 0.35212936997413635 | 0.6867794990539551 | 0.9193267822265625 | 0.6782957995951417 | 0.7180857793522267 | 0.02634741902834008
> 2 | -0.11404493451118469 | 0.32712632417678833 | 0.66926109790802 | 0.9193267822265625 | 0.8344509109311741 | 0.8610513663967612 | 0.02634741902834008
> 3 | -0.11334726959466934 | 0.32247740030288696 | 0.6625192165374756 | 0.9193267822265625 | 0.8396381578947368 | 0.8610513663967612 | 0.02634741902834008
> 4 | -0.11367174237966537 | 0.3195689618587494 | 0.6605842113494873 | 0.9193267822265625 | 0.8413777834008097 | 0.8610513663967612 | 0.02634741902834008
> 5 | -0.11365529149770737 | 0.3186545968055725 | 0.659620463848114 | 0.9193267822265625 | 0.8431806680161943 | 0.8610513663967612 | 0.02634741902834008
> 6 | -0.11342602223157883 | 0.3187546730041504 | 0.6590327024459839 | 0.9193267822265625 | 0.8439397773279352 | 0.8610513663967612 | 0.026347

### LAFTR for Eq Opp

In [11]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)    
    
    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        
        clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.07174795120954514 | 0.35173577070236206 | 0.6797263622283936 | 0.9193267822265625 | 0.6761766194331984 | 0.7128352732793523 | 0.02634741902834008
> 2 | -0.07457740604877472 | 0.32848283648490906 | 0.6694080829620361 | 0.9193267822265625 | 0.8335652834008097 | 0.8610513663967612 | 0.02634741902834008
> 3 | -0.07462367415428162 | 0.32314664125442505 | 0.6642833948135376 | 0.9193267822265625 | 0.8389739372469636 | 0.8610513663967612 | 0.02634741902834008
> 4 | -0.07471596449613571 | 0.32175302505493164 | 0.6633117198944092 | 0.9193267822265625 | 0.8409982287449392 | 0.8610513663967612 | 0.02634741902834008
> 5 | -0.0748426616191864 | 0.32056140899658203 | 0.6626993417739868 | 0.9193267822265625 | 0.8418522267206477 | 0.8610513663967612 | 0.02634741902834008
> 6 | -0.07491336762905121 | 0.3198162317276001 | 0.662277340888977 | 0.9193267822265625 | 0.8426745951417004 | 0.8610513663967612 | 0.026

## Saving into DF then CSV

In [12]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,cv_seed,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp
0,LAFTR4DP,13,0.2,0.848711,0.169891,0.796186,0.651292,0.283111,0.821610,0.737010
1,LAFTR4DP,13,0.5,0.850118,0.170980,0.808188,0.676188,0.284700,0.828623,0.753243
2,LAFTR4DP,13,0.7,0.849304,0.195629,0.826711,0.731354,0.318007,0.837855,0.785928
3,LAFTR4DP,13,1.0,0.848415,0.201663,0.829663,0.742553,0.325869,0.838934,0.791962
4,LAFTR4DP,29,0.2,0.849230,0.183348,0.802987,0.673320,0.301584,0.825461,0.751113
5,LAFTR4DP,29,0.5,0.850859,0.171463,0.801274,0.660135,0.285411,0.825322,0.743460
6,LAFTR4DP,29,0.7,0.850118,0.186897,0.819608,0.707693,0.306426,0.834584,0.772395
7,LAFTR4DP,29,1.0,0.848637,0.188628,0.809101,0.690687,0.308651,0.828398,0.761558
8,LAFTR4DP,42,0.2,0.851970,0.211024,0.815308,0.713474,0.338264,0.833236,0.776595
9,LAFTR4DP,42,0.5,0.849526,0.223781,0.815276,0.727205,0.354247,0.832049,0.783621


In [13]:
result_df.to_csv(f'{data_name}-result/laftr-{epochs}.csv')